In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import anndata as ad
import scanpy as sc
import pandas as pd
import numpy as np
import scipy.sparse as sp
import scipy.linalg
import os

import torch
used_device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [3]:
from Graspot.ST_utils import Cal_Spatial_Net, mapping_accuracy_ot, mapping_accuracy_in, batch_entropy_mixing_score, silhouette, avg_silhouette_width_batch
from Graspot.OT_utils import distance_matrix, unbalanced_ot
from Graspot.train import norm_and_center_coordinates, train_Graspot, train_Graspot_Sub

In [4]:
import scipy.sparse as sp
Batch_list = []
adj_list = []
section_ids = ['151507','151508']
#section_ids = ['151673','151674']
print(section_ids)

for section_id in section_ids:
    print(section_id)
    input_dir = os.path.join('data/', section_id)
    adata = sc.read_visium(path=input_dir, count_file=section_id + '_filtered_feature_bc_matrix.h5', load_images=True)
    adata.var_names_make_unique(join="++")

    # read the annotation
    Ann_df = pd.read_csv(os.path.join(input_dir, section_id + '_truth.txt'), sep='\t', header=None, index_col=0)
    Ann_df.columns = ['Ground Truth']
    Ann_df[Ann_df.isna()] = "unknown"
    adata.obs['Ground Truth'] = Ann_df.loc[adata.obs_names, 'Ground Truth'].astype('category')

    # make spot name unique
    adata.obs_names = [x+'_'+section_id for x in adata.obs_names]

    # Constructing the spatial network
    Cal_Spatial_Net(adata, rad_cutoff=150) # the spatial network are saved in adata.uns[‘adj’]
    # STAligner.Stats_Spatial_Net(adata) # plot the number of spatial neighbors

    # Normalization
    sc.pp.highly_variable_genes(adata, flavor="seurat_v3", n_top_genes=5000)
    sc.pp.normalize_total(adata, target_sum=1e4)
    sc.pp.log1p(adata)
    adata = adata[:, adata.var['highly_variable']]

    adj_list.append(adata.uns['adj'])
    Batch_list.append(adata)

['151507', '151508']
151507
------Calculating spatial graph...
The graph contains 24770 edges, 4226 cells.
5.8613 neighbors per cell on average.
151508
------Calculating spatial graph...
The graph contains 25698 edges, 4384 cells.
5.8618 neighbors per cell on average.


In [5]:
import anndata as ad
adata_concat = ad.concat(Batch_list, label="slice_name", keys=section_ids)
adata_concat.obs['Ground Truth'] = adata_concat.obs['Ground Truth'].astype('category')
adata_concat.obs["batch_name"] = adata_concat.obs["slice_name"].astype('category')

In [6]:
mapping_dict = {'Layer_1':1, 'Layer_2':2, 'Layer_3':3, 'Layer_4':4, 'Layer_5':5, 'Layer_6':6, 'WM':7}
data1 = np.array(Batch_list[0].obs['Ground Truth'].map(mapping_dict))
data2 = np.array(Batch_list[1].obs['Ground Truth'].map(mapping_dict))

gamma = 0.5

DM = np.ones((len(data1), len(data2)))
for i in range(len(data1)):
    for j in range(len(data2)):
        if data1[i] == data2[j]:
            DM[i][j] = gamma

In [6]:
iter_comb = [(0, 1)]
adata_concat, tran_list = train_Graspot(adata_concat, verbose=True, knn_neigh = 100, n_epochs = 200, iter_comb = iter_comb,
                                                        Batch_list=Batch_list, initial = True, device='cpu')

STAligner(
  (conv1): GATConv(2247, 512, heads=1)
  (conv2): GATConv(512, 30, heads=1)
  (conv3): GATConv(30, 512, heads=1)
  (conv4): GATConv(512, 2247, heads=1)
)
Train with STAligner...


100%|█████████████████████████████████████████| 200/200 [13:25<00:00,  4.03s/it]


In [7]:
batch_entropy_mixing_score(adata_concat.obsm['Graspot'],np.array(adata_concat.obs['batch_name']))
#0.69

0.6904598824759769

In [8]:
silhouette(adata_concat.obsm['Graspot'],np.array(adata_concat.obs['Ground Truth']))
#0.56

0.5592027194797993

In [9]:
avg_silhouette_width_batch(adata_concat.obsm['Graspot'],np.array(adata_concat.obs['batch_name']),np.array(adata_concat.obs['Ground Truth']))
#0.87

0.8748842477798462

In [7]:
iter_comb = [(0, 1)]
adata_concat, tran_list = train_Graspot(adata_concat, verbose=True, knn_neigh = 100, n_epochs = 200, iter_comb = iter_comb,
                                                        Batch_list=Batch_list, initial = True, Couple=DM, device='cpu')

STAligner(
  (conv1): GATConv(2247, 512, heads=1)
  (conv2): GATConv(512, 30, heads=1)
  (conv3): GATConv(30, 512, heads=1)
  (conv4): GATConv(512, 2247, heads=1)
)
Train with STAligner...


100%|█████████████████████████████████████████| 200/200 [13:42<00:00,  4.11s/it]


In [8]:
accuracy_ot=[]
for iters,comb in enumerate(iter_comb):
    i,j=comb[0],comb[1]
    accuracy_ot.append(mapping_accuracy_ot(Batch_list[i].obs['Ground Truth'], Batch_list[j].obs['Ground Truth'], tran_list[iters].cpu().detach().numpy()))
    print(mapping_accuracy_ot(Batch_list[i].obs['Ground Truth'], Batch_list[j].obs['Ground Truth'], tran_list[iters].cpu().detach().numpy()))

0.83078974


In [9]:
accuracy_in=[]
matching_plt=[]
getMax_plt=[]
for iters,comb in enumerate(iter_comb):
    i,j=comb[0],comb[1]
    result1 = pd.DataFrame(tran_list[iters].cpu().detach().numpy())
    if tran_list[iters].shape[0] <= tran_list[iters].shape[1]:
        matching_index = np.argmax(result1.to_numpy(),axis=1)
        matching = np.array([np.arange(result1.shape[0]),matching_index])
        getMax = np.zeros_like(tran_list[iters].cpu().detach().numpy())
        #np.put_along_axis(getMax,tran_list[iters].cpu().detach().numpy().argmax(1)[:,None],1,axis=1)
        for k in range(matching.shape[1]):
            x = int(matching[:,k][0])
            y = int(matching[:,k][1])
            getMax[x][y] = 1
        matching_plt.append(matching)
        getMax_plt.append(getMax)
    if tran_list[iters].shape[0] > tran_list[iters].shape[1]:
        matching_index = np.argmax(result1.to_numpy(),axis=0)
        matching = np.array([np.arange(result1.shape[1]),matching_index])
        getMax = np.zeros_like(tran_list[iters].cpu().detach().numpy())
        for k in range(matching.shape[1]):
            x = int(matching[:,k][1])
            y = int(matching[:,k][0])
            getMax[x][y] = 1
        matching_plt.append(matching)
        getMax_plt.append(getMax)
    
    accuracy_in.append(mapping_accuracy_in(Batch_list[i].obs['Ground Truth'], Batch_list[j].obs['Ground Truth'], getMax))
    print(mapping_accuracy_in(Batch_list[i].obs['Ground Truth'], Batch_list[j].obs['Ground Truth'], getMax))

0.8291528632276385


In [10]:
from sklearn.metrics.cluster import adjusted_rand_score
#Batch_list[1].obs['Ground Truth'][matching_plt[0][1]]
for iters,comb in enumerate(iter_comb):
    i, j = comb[0], comb[1]
    if tran_list[iters].shape[0] <= tran_list[iters].shape[1]:
        print(adjusted_rand_score(Batch_list[i].obs['Ground Truth'],Batch_list[j].obs['Ground Truth'][matching_plt[iters][1]]))#第一个小第二个大
    if tran_list[iters].shape[0] > tran_list[iters].shape[1]:
        print(adjusted_rand_score(Batch_list[j].obs['Ground Truth'],Batch_list[i].obs['Ground Truth'][matching_plt[iters][1]]))#第一个小第二个大

0.7436846717349039


In [18]:
import scipy.sparse as sp
Batch_list = []
adj_list = []
section_ids = ['151508','151509']
#section_ids = ['151673','151674']
print(section_ids)

for section_id in section_ids:
    print(section_id)
    input_dir = os.path.join('data/', section_id)
    adata = sc.read_visium(path=input_dir, count_file=section_id + '_filtered_feature_bc_matrix.h5', load_images=True)
    adata.var_names_make_unique(join="++")

    # read the annotation
    Ann_df = pd.read_csv(os.path.join(input_dir, section_id + '_truth.txt'), sep='\t', header=None, index_col=0)
    Ann_df.columns = ['Ground Truth']
    Ann_df[Ann_df.isna()] = "unknown"
    adata.obs['Ground Truth'] = Ann_df.loc[adata.obs_names, 'Ground Truth'].astype('category')

    # make spot name unique
    adata.obs_names = [x+'_'+section_id for x in adata.obs_names]

    # Constructing the spatial network
    Cal_Spatial_Net(adata, rad_cutoff=150) # the spatial network are saved in adata.uns[‘adj’]
    # STAligner.Stats_Spatial_Net(adata) # plot the number of spatial neighbors

    # Normalization
    sc.pp.highly_variable_genes(adata, flavor="seurat_v3", n_top_genes=5000)
    sc.pp.normalize_total(adata, target_sum=1e4)
    sc.pp.log1p(adata)
    adata = adata[:, adata.var['highly_variable']]

    adj_list.append(adata.uns['adj'])
    Batch_list.append(adata)

['151508', '151509']
151508
------Calculating spatial graph...
The graph contains 25698 edges, 4384 cells.
5.8618 neighbors per cell on average.
151509
------Calculating spatial graph...
The graph contains 28172 edges, 4789 cells.
5.8826 neighbors per cell on average.


In [19]:
import anndata as ad
adata_concat = ad.concat(Batch_list, label="slice_name", keys=section_ids)
adata_concat.obs['Ground Truth'] = adata_concat.obs['Ground Truth'].astype('category')
adata_concat.obs["batch_name"] = adata_concat.obs["slice_name"].astype('category')

In [20]:
mapping_dict = {'Layer_1':1, 'Layer_2':2, 'Layer_3':3, 'Layer_4':4, 'Layer_5':5, 'Layer_6':6, 'WM':7}
data1 = np.array(Batch_list[0].obs['Ground Truth'].map(mapping_dict))
data2 = np.array(Batch_list[1].obs['Ground Truth'].map(mapping_dict))

gamma = 0.5

DM = np.ones((len(data1), len(data2)))
for i in range(len(data1)):
    for j in range(len(data2)):
        if data1[i] == data2[j]:
            DM[i][j] = gamma

In [27]:
iter_comb = [(0, 1)]
adata_concat, tran_list = train_Graspot_Sub(adata_concat, verbose=True, knn_neigh = 100, n_epochs = 200, iter_comb = iter_comb,
                                                        Batch_list=Batch_list, device='cpu')

STAligner(
  (conv1): GATConv(2202, 512, heads=1)
  (conv2): GATConv(512, 30, heads=1)
  (conv3): GATConv(30, 512, heads=1)
  (conv4): GATConv(512, 2202, heads=1)
)
Pretrain with STAGATE...


100%|█████████████████████████████████████████| 200/200 [02:28<00:00,  1.34it/s]


Train with STAligner...


100%|█████████████████████████████████████████| 200/200 [15:43<00:00,  4.72s/it]


In [28]:
batch_entropy_mixing_score(adata_concat.obsm['Graspot'],np.array(adata_concat.obs['batch_name']))
#0.69

0.6896375522372031

In [29]:
silhouette(adata_concat.obsm['Graspot'],np.array(adata_concat.obs['Ground Truth']))
#0.48

0.47644619457423687

In [30]:
avg_silhouette_width_batch(adata_concat.obsm['Graspot'],np.array(adata_concat.obs['batch_name']),np.array(adata_concat.obs['Ground Truth']))
#0.77

0.7775200009346008

In [21]:
iter_comb = [(0, 1)]
adata_concat, tran_list = train_Graspot_Sub(adata_concat, verbose=True, knn_neigh = 100, n_epochs = 200, iter_comb = iter_comb,
                                                        Batch_list=Batch_list, Couple=DM, device='cpu')

STAligner(
  (conv1): GATConv(2202, 512, heads=1)
  (conv2): GATConv(512, 30, heads=1)
  (conv3): GATConv(30, 512, heads=1)
  (conv4): GATConv(512, 2202, heads=1)
)
Pretrain with STAGATE...


100%|█████████████████████████████████████████| 200/200 [02:34<00:00,  1.29it/s]


Train with STAligner...


100%|█████████████████████████████████████████| 200/200 [15:34<00:00,  4.67s/it]


In [22]:
#有prior
accuracy_ot=[]
for iters,comb in enumerate(iter_comb):
    i,j=comb[0],comb[1]
    accuracy_ot.append(mapping_accuracy_ot(Batch_list[i].obs['Ground Truth'], Batch_list[j].obs['Ground Truth'], tran_list[iters].cpu().detach().numpy()))
    print(mapping_accuracy_ot(Batch_list[i].obs['Ground Truth'], Batch_list[j].obs['Ground Truth'], tran_list[iters].cpu().detach().numpy()))

0.77952147


In [23]:
#有prior
accuracy_in=[]
matching_plt=[]
getMax_plt=[]
for iters,comb in enumerate(iter_comb):
    i,j=comb[0],comb[1]
    result1 = pd.DataFrame(tran_list[iters].cpu().detach().numpy())
    if tran_list[iters].shape[0] <= tran_list[iters].shape[1]:
        matching_index = np.argmax(result1.to_numpy(),axis=1)
        matching = np.array([np.arange(result1.shape[0]),matching_index])
        getMax = np.zeros_like(tran_list[iters].cpu().detach().numpy())
        #np.put_along_axis(getMax,tran_list[iters].cpu().detach().numpy().argmax(1)[:,None],1,axis=1)
        for k in range(matching.shape[1]):
            x = int(matching[:,k][0])
            y = int(matching[:,k][1])
            getMax[x][y] = 1
        matching_plt.append(matching)
        getMax_plt.append(getMax)
    if tran_list[iters].shape[0] > tran_list[iters].shape[1]:
        matching_index = np.argmax(result1.to_numpy(),axis=0)
        matching = np.array([np.arange(result1.shape[1]),matching_index])
        getMax = np.zeros_like(tran_list[iters].cpu().detach().numpy())
        for k in range(matching.shape[1]):
            x = int(matching[:,k][1])
            y = int(matching[:,k][0])
            getMax[x][y] = 1
        matching_plt.append(matching)
        getMax_plt.append(getMax)
    
    accuracy_in.append(mapping_accuracy_in(Batch_list[i].obs['Ground Truth'], Batch_list[j].obs['Ground Truth'], getMax))
    print(mapping_accuracy_in(Batch_list[i].obs['Ground Truth'], Batch_list[j].obs['Ground Truth'], getMax))

0.771441605839416


In [24]:
#有prior
from sklearn.metrics.cluster import adjusted_rand_score
#Batch_list[1].obs['Ground Truth'][matching_plt[0][1]]
for iters,comb in enumerate(iter_comb):
    i, j = comb[0], comb[1]
    if tran_list[iters].shape[0] <= tran_list[iters].shape[1]:
        print(adjusted_rand_score(Batch_list[i].obs['Ground Truth'],Batch_list[j].obs['Ground Truth'][matching_plt[iters][1]]))#第一个小第二个大
    if tran_list[iters].shape[0] > tran_list[iters].shape[1]:
        print(adjusted_rand_score(Batch_list[j].obs['Ground Truth'],Batch_list[i].obs['Ground Truth'][matching_plt[iters][1]]))#第一个小第二个大

0.5717359473267548


In [31]:
accuracy_ot=[]
for iters,comb in enumerate(iter_comb):
    i,j=comb[0],comb[1]
    accuracy_ot.append(mapping_accuracy_ot(Batch_list[i].obs['Ground Truth'], Batch_list[j].obs['Ground Truth'], tran_list[iters].cpu().detach().numpy()))
    print(mapping_accuracy_ot(Batch_list[i].obs['Ground Truth'], Batch_list[j].obs['Ground Truth'], tran_list[iters].cpu().detach().numpy()))

0.56969315


In [32]:
accuracy_in=[]
matching_plt=[]
getMax_plt=[]
for iters,comb in enumerate(iter_comb):
    i,j=comb[0],comb[1]
    result1 = pd.DataFrame(tran_list[iters].cpu().detach().numpy())
    if tran_list[iters].shape[0] < tran_list[iters].shape[1]:
        matching_index = np.argmax(result1.to_numpy(),axis=1)
        matching = np.array([np.arange(result1.shape[0]),matching_index])
        getMax = np.zeros_like(tran_list[iters].cpu().detach().numpy())
        #np.put_along_axis(getMax,tran_list[iters].cpu().detach().numpy().argmax(1)[:,None],1,axis=1)
        for k in range(matching.shape[1]):
            x = int(matching[:,k][0])
            y = int(matching[:,k][1])
            getMax[x][y] = 1
        matching_plt.append(matching)
        getMax_plt.append(getMax)
    if tran_list[iters].shape[0] > tran_list[iters].shape[1]:
        matching_index = np.argmax(result1.to_numpy(),axis=0)
        matching = np.array([np.arange(result1.shape[1]),matching_index])
        getMax = np.zeros_like(tran_list[iters].cpu().detach().numpy())
        for k in range(matching.shape[1]):
            x = int(matching[:,k][1])
            y = int(matching[:,k][0])
            getMax[x][y] = 1
        matching_plt.append(matching)
        getMax_plt.append(getMax)
    
    accuracy_in.append(mapping_accuracy_in(Batch_list[i].obs['Ground Truth'], Batch_list[j].obs['Ground Truth'], getMax))
    print(mapping_accuracy_in(Batch_list[i].obs['Ground Truth'], Batch_list[j].obs['Ground Truth'], getMax))

0.5716240875912408


In [33]:
from sklearn.metrics.cluster import adjusted_rand_score
#Batch_list[1].obs['Ground Truth'][matching_plt[0][1]]
for iters,comb in enumerate(iter_comb):
    i, j = comb[0], comb[1]
    if tran_list[iters].shape[0] <= tran_list[iters].shape[1]:
        print(adjusted_rand_score(Batch_list[i].obs['Ground Truth'],Batch_list[j].obs['Ground Truth'][matching_plt[iters][1]]))#第一个小第二个大
    if tran_list[iters].shape[0] > tran_list[iters].shape[1]:
        print(adjusted_rand_score(Batch_list[j].obs['Ground Truth'],Batch_list[i].obs['Ground Truth'][matching_plt[iters][1]]))#第一个小第二个大

0.356707833596431


In [11]:
import scipy.sparse as sp
Batch_list = []
adj_list = []
section_ids = ['151509','151510']
#section_ids = ['151673','151674']
print(section_ids)

for section_id in section_ids:
    print(section_id)
    input_dir = os.path.join('data/', section_id)
    adata = sc.read_visium(path=input_dir, count_file=section_id + '_filtered_feature_bc_matrix.h5', load_images=True)
    adata.var_names_make_unique(join="++")

    # read the annotation
    Ann_df = pd.read_csv(os.path.join(input_dir, section_id + '_truth.txt'), sep='\t', header=None, index_col=0)
    Ann_df.columns = ['Ground Truth']
    Ann_df[Ann_df.isna()] = "unknown"
    adata.obs['Ground Truth'] = Ann_df.loc[adata.obs_names, 'Ground Truth'].astype('category')

    # make spot name unique
    adata.obs_names = [x+'_'+section_id for x in adata.obs_names]

    # Constructing the spatial network
    Cal_Spatial_Net(adata, rad_cutoff=150) # the spatial network are saved in adata.uns[‘adj’]
    # STAligner.Stats_Spatial_Net(adata) # plot the number of spatial neighbors

    # Normalization
    sc.pp.highly_variable_genes(adata, flavor="seurat_v3", n_top_genes=5000)
    sc.pp.normalize_total(adata, target_sum=1e4)
    sc.pp.log1p(adata)
    adata = adata[:, adata.var['highly_variable']]

    adj_list.append(adata.uns['adj'])
    Batch_list.append(adata)

['151509', '151510']
151509
------Calculating spatial graph...
The graph contains 28172 edges, 4789 cells.
5.8826 neighbors per cell on average.
151510
------Calculating spatial graph...
The graph contains 27198 edges, 4634 cells.
5.8692 neighbors per cell on average.


In [12]:
import anndata as ad
adata_concat = ad.concat(Batch_list, label="slice_name", keys=section_ids)
adata_concat.obs['Ground Truth'] = adata_concat.obs['Ground Truth'].astype('category')
adata_concat.obs["batch_name"] = adata_concat.obs["slice_name"].astype('category')

In [13]:
mapping_dict = {'Layer_1':1, 'Layer_2':2, 'Layer_3':3, 'Layer_4':4, 'Layer_5':5, 'Layer_6':6, 'WM':7}
data1 = np.array(Batch_list[0].obs['Ground Truth'].map(mapping_dict))
data2 = np.array(Batch_list[1].obs['Ground Truth'].map(mapping_dict))

gamma = 0.5

DM = np.ones((len(data1), len(data2)))
for i in range(len(data1)):
    for j in range(len(data2)):
        if data1[i] == data2[j]:
            DM[i][j] = gamma

In [21]:
iter_comb = [(0, 1)]
adata_concat, tran_list = train_Graspot(adata_concat, verbose=True, knn_neigh = 100, n_epochs = 200, iter_comb = iter_comb,
                                                        Batch_list=Batch_list, initial = True, device='cpu')

STAligner(
  (conv1): GATConv(2356, 512, heads=1)
  (conv2): GATConv(512, 30, heads=1)
  (conv3): GATConv(30, 512, heads=1)
  (conv4): GATConv(512, 2356, heads=1)
)
Train with STAligner...


100%|█████████████████████████████████████████| 200/200 [16:43<00:00,  5.02s/it]


In [22]:
batch_entropy_mixing_score(adata_concat.obsm['Graspot'],np.array(adata_concat.obs['batch_name']))
#0.69

0.6905071033776377

In [23]:
silhouette(adata_concat.obsm['Graspot'],np.array(adata_concat.obs['Ground Truth']))
#0.59

0.5860288143157959

In [24]:
avg_silhouette_width_batch(adata_concat.obsm['Graspot'],np.array(adata_concat.obs['batch_name']),np.array(adata_concat.obs['Ground Truth']))
#0.89

0.8939626812934875

In [14]:
iter_comb = [(0, 1)]
adata_concat, tran_list = train_Graspot(adata_concat, verbose=True, knn_neigh = 100, n_epochs = 200, iter_comb = iter_comb,
                                                        Batch_list=Batch_list, initial = True, Couple=DM, device='cpu')

STAligner(
  (conv1): GATConv(2356, 512, heads=1)
  (conv2): GATConv(512, 30, heads=1)
  (conv3): GATConv(30, 512, heads=1)
  (conv4): GATConv(512, 2356, heads=1)
)
Train with STAligner...


100%|█████████████████████████████████████████| 200/200 [16:34<00:00,  4.97s/it]


In [15]:
accuracy_ot=[]
for iters,comb in enumerate(iter_comb):
    i,j=comb[0],comb[1]
    accuracy_ot.append(mapping_accuracy_ot(Batch_list[i].obs['Ground Truth'], Batch_list[j].obs['Ground Truth'], tran_list[iters].cpu().detach().numpy()))
    print(mapping_accuracy_ot(Batch_list[i].obs['Ground Truth'], Batch_list[j].obs['Ground Truth'], tran_list[iters].cpu().detach().numpy()))

0.85524666


In [16]:
accuracy_in=[]
matching_plt=[]
getMax_plt=[]
for iters,comb in enumerate(iter_comb):
    i,j=comb[0],comb[1]
    result1 = pd.DataFrame(tran_list[iters].cpu().detach().numpy())
    if tran_list[iters].shape[0] <= tran_list[iters].shape[1]:
        matching_index = np.argmax(result1.to_numpy(),axis=1)
        matching = np.array([np.arange(result1.shape[0]),matching_index])
        getMax = np.zeros_like(tran_list[iters].cpu().detach().numpy())
        #np.put_along_axis(getMax,tran_list[iters].cpu().detach().numpy().argmax(1)[:,None],1,axis=1)
        for k in range(matching.shape[1]):
            x = int(matching[:,k][0])
            y = int(matching[:,k][1])
            getMax[x][y] = 1
        matching_plt.append(matching)
        getMax_plt.append(getMax)
    if tran_list[iters].shape[0] > tran_list[iters].shape[1]:
        matching_index = np.argmax(result1.to_numpy(),axis=0)
        matching = np.array([np.arange(result1.shape[1]),matching_index])
        getMax = np.zeros_like(tran_list[iters].cpu().detach().numpy())
        for k in range(matching.shape[1]):
            x = int(matching[:,k][1])
            y = int(matching[:,k][0])
            getMax[x][y] = 1
        matching_plt.append(matching)
        getMax_plt.append(getMax)
    
    accuracy_in.append(mapping_accuracy_in(Batch_list[i].obs['Ground Truth'], Batch_list[j].obs['Ground Truth'], getMax))
    print(mapping_accuracy_in(Batch_list[i].obs['Ground Truth'], Batch_list[j].obs['Ground Truth'], getMax))

0.853258523953388


In [17]:
from sklearn.metrics.cluster import adjusted_rand_score
#Batch_list[1].obs['Ground Truth'][matching_plt[0][1]]
for iters,comb in enumerate(iter_comb):
    i, j = comb[0], comb[1]
    if tran_list[iters].shape[0] <= tran_list[iters].shape[1]:
        print(adjusted_rand_score(Batch_list[i].obs['Ground Truth'],Batch_list[j].obs['Ground Truth'][matching_plt[iters][1]]))#第一个小第二个大
    if tran_list[iters].shape[0] > tran_list[iters].shape[1]:
        print(adjusted_rand_score(Batch_list[j].obs['Ground Truth'],Batch_list[i].obs['Ground Truth'][matching_plt[iters][1]]))#第一个小第二个大

0.7186730118454987


In [4]:
import scipy.sparse as sp
Batch_list = []
adj_list = []
section_ids = ['151507','151508','151509','151510']
#section_ids = ['151673','151674']
print(section_ids)

for section_id in section_ids:
    print(section_id)
    input_dir = os.path.join('data/', section_id)
    adata = sc.read_visium(path=input_dir, count_file=section_id + '_filtered_feature_bc_matrix.h5', load_images=True)
    adata.var_names_make_unique(join="++")

    # read the annotation
    Ann_df = pd.read_csv(os.path.join(input_dir, section_id + '_truth.txt'), sep='\t', header=None, index_col=0)
    Ann_df.columns = ['Ground Truth']
    Ann_df[Ann_df.isna()] = "unknown"
    adata.obs['Ground Truth'] = Ann_df.loc[adata.obs_names, 'Ground Truth'].astype('category')

    # make spot name unique
    adata.obs_names = [x+'_'+section_id for x in adata.obs_names]

    # Constructing the spatial network
    Cal_Spatial_Net(adata, rad_cutoff=150) # the spatial network are saved in adata.uns[‘adj’]
    # STAligner.Stats_Spatial_Net(adata) # plot the number of spatial neighbors

    # Normalization
    sc.pp.highly_variable_genes(adata, flavor="seurat_v3", n_top_genes=5000)
    sc.pp.normalize_total(adata, target_sum=1e4)
    sc.pp.log1p(adata)
    adata = adata[:, adata.var['highly_variable']]

    adj_list.append(adata.uns['adj'])
    Batch_list.append(adata)

['151507', '151508', '151509', '151510']
151507
------Calculating spatial graph...
The graph contains 24770 edges, 4226 cells.
5.8613 neighbors per cell on average.
151508
------Calculating spatial graph...
The graph contains 25698 edges, 4384 cells.
5.8618 neighbors per cell on average.
151509
------Calculating spatial graph...
The graph contains 28172 edges, 4789 cells.
5.8826 neighbors per cell on average.
151510
------Calculating spatial graph...
The graph contains 27198 edges, 4634 cells.
5.8692 neighbors per cell on average.


In [6]:
import anndata as ad
adata_concat = ad.concat(Batch_list, label="slice_name", keys=section_ids)
adata_concat.obs['Ground Truth'] = adata_concat.obs['Ground Truth'].astype('category')
adata_concat.obs["batch_name"] = adata_concat.obs["slice_name"].astype('category')

In [7]:
iter_comb = [(0, 1), (2, 3)]
adata_concat, tran_list = train_Graspot(adata_concat, verbose=True, knn_neigh = 100, n_epochs = 200, iter_comb = iter_comb,
                                                        Batch_list=Batch_list, initial = True, device='cpu')
#adata_concat, tran_list = train_Graspot_Sub(adata_concat, verbose=True, knn_neigh = 100, n_epochs = 200, iter_comb = iter_comb,
                                                        #Batch_list=Batch_list,  initial = True, device=used_device)

STAligner(
  (conv1): GATConv(1064, 512, heads=1)
  (conv2): GATConv(512, 30, heads=1)
  (conv3): GATConv(30, 512, heads=1)
  (conv4): GATConv(512, 1064, heads=1)
)
Train with STAligner...


100%|████████████████████████████████████| 200/200 [14:48<00:00,  4.44s/it]


In [8]:
accuracy_ot=[]
for iters,comb in enumerate(iter_comb):
    i,j=comb[0],comb[1]
    accuracy_ot.append(mapping_accuracy_ot(Batch_list[i].obs['Ground Truth'], Batch_list[j].obs['Ground Truth'], tran_list[iters].cpu().detach().numpy()))
    print(mapping_accuracy_ot(Batch_list[i].obs['Ground Truth'], Batch_list[j].obs['Ground Truth'], tran_list[iters].cpu().detach().numpy()))

0.8005708
0.84505504


In [9]:
accuracy_in=[]
matching_plt=[]
getMax_plt=[]
for iters,comb in enumerate(iter_comb):
    i,j=comb[0],comb[1]
    result1 = pd.DataFrame(tran_list[iters].cpu().detach().numpy())
    if tran_list[iters].shape[0] < tran_list[iters].shape[1]:
        matching_index = np.argmax(result1.to_numpy(),axis=1)
        matching = np.array([np.arange(result1.shape[0]),matching_index])
        getMax = np.zeros_like(tran_list[iters].cpu().detach().numpy())
        #np.put_along_axis(getMax,tran_list[iters].cpu().detach().numpy().argmax(1)[:,None],1,axis=1)
        for k in range(matching.shape[1]):
            x = int(matching[:,k][0])
            y = int(matching[:,k][1])
            getMax[x][y] = 1
        matching_plt.append(matching)
        getMax_plt.append(getMax)
    if tran_list[iters].shape[0] > tran_list[iters].shape[1]:
        matching_index = np.argmax(result1.to_numpy(),axis=0)
        matching = np.array([np.arange(result1.shape[1]),matching_index])
        getMax = np.zeros_like(tran_list[iters].cpu().detach().numpy())
        for k in range(matching.shape[1]):
            x = int(matching[:,k][1])
            y = int(matching[:,k][0])
            getMax[x][y] = 1
        matching_plt.append(matching)
        getMax_plt.append(getMax)
    
    accuracy_in.append(mapping_accuracy_in(Batch_list[i].obs['Ground Truth'], Batch_list[j].obs['Ground Truth'], getMax))
    print(mapping_accuracy_in(Batch_list[i].obs['Ground Truth'], Batch_list[j].obs['Ground Truth'], getMax))

0.8024136299100805
0.8452740612861459


In [10]:
from sklearn.metrics.cluster import adjusted_rand_score
#Batch_list[1].obs['Ground Truth'][matching_plt[0][1]]
for iters,comb in enumerate(iter_comb):
    i, j = comb[0], comb[1]
    if tran_list[iters].shape[0] < tran_list[iters].shape[1]:
        print(adjusted_rand_score(Batch_list[i].obs['Ground Truth'],Batch_list[j].obs['Ground Truth'][matching_plt[iters][1]]))#第一个小第二个大
    if tran_list[iters].shape[0] > tran_list[iters].shape[1]:
        print(adjusted_rand_score(Batch_list[j].obs['Ground Truth'],Batch_list[i].obs['Ground Truth'][matching_plt[iters][1]]))#第一个小第二个大

0.7057775055437974
0.7093921476892171


In [11]:
import scipy.sparse as sp
Batch_list = []
adj_list = []
section_ids = ['151508','151509']
#section_ids = ['151673','151674']
print(section_ids)

for section_id in section_ids:
    print(section_id)
    input_dir = os.path.join('data/', section_id)
    adata = sc.read_visium(path=input_dir, count_file=section_id + '_filtered_feature_bc_matrix.h5', load_images=True)
    adata.var_names_make_unique(join="++")

    # read the annotation
    Ann_df = pd.read_csv(os.path.join(input_dir, section_id + '_truth.txt'), sep='\t', header=None, index_col=0)
    Ann_df.columns = ['Ground Truth']
    Ann_df[Ann_df.isna()] = "unknown"
    adata.obs['Ground Truth'] = Ann_df.loc[adata.obs_names, 'Ground Truth'].astype('category')

    # make spot name unique
    adata.obs_names = [x+'_'+section_id for x in adata.obs_names]

    # Constructing the spatial network
    Cal_Spatial_Net(adata, rad_cutoff=150) # the spatial network are saved in adata.uns[‘adj’]
    # STAligner.Stats_Spatial_Net(adata) # plot the number of spatial neighbors

    # Normalization
    sc.pp.highly_variable_genes(adata, flavor="seurat_v3", n_top_genes=5000)
    sc.pp.normalize_total(adata, target_sum=1e4)
    sc.pp.log1p(adata)
    adata = adata[:, adata.var['highly_variable']]

    adj_list.append(adata.uns['adj'])
    Batch_list.append(adata)

['151508', '151509']
151508
------Calculating spatial graph...
The graph contains 25698 edges, 4384 cells.
5.8618 neighbors per cell on average.
151509
------Calculating spatial graph...
The graph contains 28172 edges, 4789 cells.
5.8826 neighbors per cell on average.


In [12]:
import anndata as ad
adata_concat = ad.concat(Batch_list, label="slice_name", keys=section_ids)
adata_concat.obs['Ground Truth'] = adata_concat.obs['Ground Truth'].astype('category')
adata_concat.obs["batch_name"] = adata_concat.obs["slice_name"].astype('category')

In [13]:
iter_comb = [(0, 1)]
adata_concat, tran_list = train_Graspot_Sub(adata_concat, verbose=True, knn_neigh = 100, n_epochs = 200, iter_comb = iter_comb,
                                                        Batch_list=Batch_list, device='cpu')

STAligner(
  (conv1): GATConv(2202, 512, heads=1)
  (conv2): GATConv(512, 30, heads=1)
  (conv3): GATConv(30, 512, heads=1)
  (conv4): GATConv(512, 2202, heads=1)
)
Pretrain with STAGATE...


100%|████████████████████████████████████| 200/200 [02:31<00:00,  1.32it/s]


Train with STAligner...


100%|████████████████████████████████████| 200/200 [14:19<00:00,  4.30s/it]


In [14]:
accuracy_ot=[]
for iters,comb in enumerate(iter_comb):
    i,j=comb[0],comb[1]
    accuracy_ot.append(mapping_accuracy_ot(Batch_list[i].obs['Ground Truth'], Batch_list[j].obs['Ground Truth'], tran_list[iters].cpu().detach().numpy()))
    print(mapping_accuracy_ot(Batch_list[i].obs['Ground Truth'], Batch_list[j].obs['Ground Truth'], tran_list[iters].cpu().detach().numpy()))

0.56969315


In [15]:
accuracy_in=[]
matching_plt=[]
getMax_plt=[]
for iters,comb in enumerate(iter_comb):
    i,j=comb[0],comb[1]
    result1 = pd.DataFrame(tran_list[iters].cpu().detach().numpy())
    if tran_list[iters].shape[0] < tran_list[iters].shape[1]:
        matching_index = np.argmax(result1.to_numpy(),axis=1)
        matching = np.array([np.arange(result1.shape[0]),matching_index])
        getMax = np.zeros_like(tran_list[iters].cpu().detach().numpy())
        #np.put_along_axis(getMax,tran_list[iters].cpu().detach().numpy().argmax(1)[:,None],1,axis=1)
        for k in range(matching.shape[1]):
            x = int(matching[:,k][0])
            y = int(matching[:,k][1])
            getMax[x][y] = 1
        matching_plt.append(matching)
        getMax_plt.append(getMax)
    if tran_list[iters].shape[0] > tran_list[iters].shape[1]:
        matching_index = np.argmax(result1.to_numpy(),axis=0)
        matching = np.array([np.arange(result1.shape[1]),matching_index])
        getMax = np.zeros_like(tran_list[iters].cpu().detach().numpy())
        for k in range(matching.shape[1]):
            x = int(matching[:,k][1])
            y = int(matching[:,k][0])
            getMax[x][y] = 1
        matching_plt.append(matching)
        getMax_plt.append(getMax)
    
    accuracy_in.append(mapping_accuracy_in(Batch_list[i].obs['Ground Truth'], Batch_list[j].obs['Ground Truth'], getMax))
    print(mapping_accuracy_in(Batch_list[i].obs['Ground Truth'], Batch_list[j].obs['Ground Truth'], getMax))

0.5716240875912408


In [16]:
from sklearn.metrics.cluster import adjusted_rand_score
#Batch_list[1].obs['Ground Truth'][matching_plt[0][1]]
for iters,comb in enumerate(iter_comb):
    i, j = comb[0], comb[1]
    if tran_list[iters].shape[0] < tran_list[iters].shape[1]:
        print(adjusted_rand_score(Batch_list[i].obs['Ground Truth'],Batch_list[j].obs['Ground Truth'][matching_plt[iters][1]]))#第一个小第二个大
    if tran_list[iters].shape[0] > tran_list[iters].shape[1]:
        print(adjusted_rand_score(Batch_list[j].obs['Ground Truth'],Batch_list[i].obs['Ground Truth'][matching_plt[iters][1]]))#第一个小第二个大

0.356707833596431


In [13]:
accuracy_in=[]
for iters,comb in enumerate(iter_comb):
    i,j=comb[0],comb[1]
    result1 = pd.DataFrame(tran_list[iters].cpu().detach().numpy())
    matching_index = np.argmax(result1.to_numpy(),axis=0)
    matching = np.array([np.arange(result1.shape[1]),matching_index])
    getMax = np.zeros_like(tran_list[iters].cpu().detach().numpy())
    np.put_along_axis(getMax,tran_list[iters].cpu().detach().numpy().argmax(1)[:,None],1,axis=1)
    
    accuracy_in.append(mapping_accuracy_in(Batch_list[i].obs['Ground Truth'], Batch_list[j].obs['Ground Truth'], getMax))
    print(mapping_accuracy_in(Batch_list[i].obs['Ground Truth'], Batch_list[j].obs['Ground Truth'], getMax))

0.5716240875912408
